# Fund Data Analysis and Visualization

### Importing Required Libraries

In [ ]:
import pandas as pd
from typing import List, Dict
import matplotlib.pyplot as plt
import seaborn as sns

### Loading and Exploring Fund Data

In [ ]:
root: str = "./files/Funds.xlsx"
fund = pd.read_excel(root,parse_dates=["Date"], date_format='%m/%d/%Y') # Transforming DATE : correct format
fund.head(5)

### Dataframe Information

In [ ]:
fund.info()

### Data Cleaning - Share Price Conversion

In [ ]:

fund["Share_Price"] = [ (lambda x: float(x.replace("$",(""))))(num) for num in fund["Share_Price"] ]
fund.head(5)

### Unique Fund Symbols Analysis

In [ ]:
unique_values = fund["Symbol"].unique()
for x in unique_values:
    print(x)

### Data Analysis and Visualization Functions

In [ ]:
months = [
    'January', 'February', 'March', 'April', 
    'May', 'June', 'July', 'August', 
    'September', 'October', 'November', 'December'
]

def plot_share_price_simple(df: pd.DataFrame, company_name: str, color: str, months: List[str] = months) -> None:
    """
    Create a bar chart showing mean share price by month for a specific fund.
    
    Args:
        df: DataFrame containing fund data
        company_name: Name of the fund for title
        color: Color for the bars
        months: List of month names in order
    """
    # Calculate mean values by month
    data = df.groupby(df["Date"].dt.month_name())["Share_Price"].agg("mean")
    data.index = pd.Categorical(data.index, categories=months, ordered=True) # type: ignore
    data = data.sort_index()
    
    y = data.values
    x = data.index

    # Create visualization
    plt.figure(figsize=(7, 5))

    # Add value labels on bars
    for i, total in enumerate(y):
        plt.text(i, total + 0.1, f'${total:.2f}',
                 ha='center', weight='bold', color='black')

    plt.bar(x, y, color=color)
    plt.ylabel('Price $ USD')
    plt.title(f'{company_name} - Mean Share Price by Month')
    plt.xticks(rotation=30)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return

def get_fund_monthly_stats(df: pd.DataFrame, fund_name: str, months: List[str] = months) -> pd.DataFrame:
    """
    Calculate and display monthly statistics for a fund's share price.
    
    Args:
        df: DataFrame containing fund data
        fund_name: Name of the fund for reporting
        months: List of month names in order
        
    Returns:
        DataFrame with monthly statistics
    """
    agg_data = ["count", "median", "mean", "max", "min"]
    
    # Calculate statistics by month
    data = df.groupby(df["Date"].dt.month_name())["Share_Price"].agg(agg_data) # type: ignore
    data.index = pd.Categorical(data.index, categories=months, ordered=True)
    data = data.sort_index()
    
    print(f"=== MONTHLY STATISTICS - {fund_name} ===")
    print(data.round(3))
    
    return data

def get_fund_info(df: pd.DataFrame, fund_name: str) -> pd.DataFrame:
    """
    Filter dataframe to get data for a specific fund.
    
    Args:
        df: Main DataFrame containing all funds data
        fund_name: Symbol of the fund to filter
        
    Returns:
        Filtered DataFrame with only the specified fund's data
    """
    _filter = df["Symbol"] == fund_name
    data = df[_filter]
    return data
 

## ACP Fund Analysis

### Filtering ACP Fund Data

In [ ]:
ACPDF = get_fund_info(fund,"ACP")
ACPDF.head(5)

### Monthly Statistics for ACP Fund

In [ ]:
acp_stats = get_fund_monthly_stats(ACPDF, "ACP")

### ACP Fund Share Price Visualization

In [ ]:
plot_share_price_simple(ACPDF,"ACP","#2E86AB")

## FOF Fund Analysis

### Filtering FOF Fund Data

In [ ]:
FOFDF = get_fund_info(fund,"FOF")
FOFDF.head(5)

### Monthly Statistics for FOF Fund

In [ ]:
fof_stats = get_fund_monthly_stats(FOFDF, "FOF")

### FOF Fund Share Price Visualization

In [ ]:
plot_share_price_simple(FOFDF,"FOF","#A23B72")

## EVT Fund Analysis

### Filtering EVT Fund Data 

In [ ]:
EVTDF = get_fund_info(fund,"EVT")
EVTDF.head(5)

### Monthly Statistics for EVT Fund

In [ ]:
evt_stats = get_fund_monthly_stats(EVTDF, "EVT")

### EVT Fund Share Price Visualization

In [ ]:
plot_share_price_simple(EVTDF,"EVT","#F18F01")